### Day - 1 : Practice Questions

In [0]:
dbutils.fs.ls('/FileStore/tables/')

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Q. 1 - Read Data : Load the dataset from CSV and JSON into a PySpark DataFrame.

In [0]:
df = spark.read.format('json')\
    .option('InferSchema', True)\
        .option('header', True)\
            .option('multiLine', False)\
            .load('/FileStore/tables/employees-1.json')

In [0]:
df.display()

### Q.2 - Select & Alias :

### 1. Select only first_name, last_name, and salary.

### 2. Create a column full_name using alias.

In [0]:
df.select(col('first_name'),col('last_name'),col('salary')).display()

In [0]:
df.withColumn("FullName", concat_ws(" ",col('first_name'),col('last_name'))).display()

In [0]:
df.select(col('first_name')).display()

### Q.3 - Filter :

### 1. Find employees with salary > 70000.

### 2. Find employees from the IT department who are younger than 35.

In [0]:
df.filter(col('salary')>70000).display()

In [0]:
df.filter((col('department') =='IT') & (col('age')<35)).display()

### Q.4 - withColumnRenamed

### Rename column first_name → fname and last_name → lname.

In [0]:
df.withColumnRenamed('first_name','fname')\
    .withColumnRenamed('last_name','lname').display()

### Q.5 - withColumn :

### Add a new column bonus = 10% of salary.

### Add a column experience = number of years since joining_date.

In [0]:
df.withColumn('bonus', 0.10 * col('salary')).display()

In [0]:
df.withColumn("Experience", datediff(current_date(),col('joining_date'))/365).sort(col('experience')).display()

### Q.6 - Type Casting

### 1.Cast salary from double → integer.

### 2.Cast joining_date from string → date.

In [0]:
df.withColumn('salary',col('salary').cast(IntegerType())).display()
    

In [0]:
df.withColumn('joining_date',col('joining_date').cast(DateType())).display()

### Q.7 - Sort :

### 1. Sort employees by salary descending.

### 2. Sort employees by department and then by age.

In [0]:
df.sort(col('salary').desc()).display()

In [0]:
df.sort('department','age',ascending = [1,1]).display()

### Q.8 - Limit :

### Show top 5 highest-paid employees.

In [0]:
df.sort(col('salary').desc()).limit(5).display()

### Q.9 - Drop Duplicates :

### Suppose duplicate rows appear, drop them based on emp_id.

In [0]:
df.drop_duplicates(subset=['emp_id']).display()

### Q. 11 - Select & Alias

###Select emp_id, department, and salary but show them as EmployeeID, Dept, and AnnualSalary.

In [0]:
df.select(col('emp_id').alias('EmployeeID'), col('department').alias('Dept'),col('Salary').alias('AnnualSalary')).display()

### Q. 12 - Filter :

### 1. Find employees in Finance or Marketing departments with salary greater than 75000.

### 2. Find employees whose city is not New York and age < 35.

In [0]:
df.filter((col('department').isin('Marketing','Finance')) & (col('salary')>75000)).display()

In [0]:
df.filter((col('city') != 'New York') & (col('age')<35)).sort('age').display()

### Q.13. withColumnRenamed :

### Rename emp_id → employee_id and joining_date → join_date.

In [0]:
df.withColumnRenamed('emp_id','employee_id').withColumnRenamed('joining_date','join_date').display()

### 14. withColumn

### Add a new column monthly_salary = salary / 12.

### Add a column senior_flag which is "Yes" if age ≥ 40, else "No".

### Add a column city_upper which contains the city name in uppercase.

In [0]:
df.withColumn('monthly_salary', col('salary')/12).display()

### Q.15. Type Casting

### 1.Cast emp_id to string.

### 2.Cast age to double.

In [0]:
df.withColumn('emp_id', col('emp_id').cast(StringType())).display()

In [0]:
df.withColumn('age', col('age').cast(DoubleType())).display()

### 16. Sort -

### 1. Sort employees by joining_date (earliest first).

### 2. Sort employees by department, then descending salary.

In [0]:
df.withColumn('joining_date', col('joining_date')).sort(col('joining_date')).display()

In [0]:
df.sort(['department','salary'],ascending=[0,0]).display()

### 17. Limit :

### 1. Show top 3 youngest employees.

### 2. Show top 2 employees from the HR department with the highest salary.

In [0]:
df.withColumn('age', col('age')).sort(col('age')).limit(3).display()

In [0]:
df.filter(col('department')=='HR').sort(col('salary').desc()).limit(2).display()

### 18. Drop Duplicates :

### Suppose you join data from two sources and end up with duplicates — drop duplicates based only on first_name, last_name, and city.

In [0]:
df.drop_duplicates(subset=('first_name','last_name','city')).display()

### 19. Bonus Challenge (Combo) :

### Create a column emp_info which concatenates:
### "John Smith - IT - New York" (format: full_name - department - city).

### From this result, sort employees by length of emp_info.

In [0]:
df_fullname = df.withColumn('full_name', concat_ws(' ',col('first_name'),col('last_name')))

In [0]:
df_empinfo = df_fullname.withColumn('emp_info', concat_ws('-', col('full_name'), col('department'), col('city')))

In [0]:
df_empinfo.display()

In [0]:
df_empinfo.withColumn('len_emp_info', length( col('emp_info'))).sort('len_emp_info').display()

In [0]:
dbutils.fs.ls('/FileStore/tables')

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
df = spark.read.format('csv')\
    .option('InferSchema',True)\
        .option('header',True)\
            .load('/FileStore/tables/employees.csv')

In [0]:
df.display()

### 20. Select & Alias

### 1. Select first_name, last_name, and city, but show them as FName, LName, and Location.

### 2. Select department and count the number of employees in each department (alias as DeptCount).

In [0]:
df.select(col('first_name').alias('FName'), col('last_name').alias('LName'), col('city').alias('Location')).display()

In [0]:
df.select(col('department')).groupBy(col('department')).agg(count('*').alias('DeptCount')).display()

### 21. Filter :

### 1. Get all employees who joined before 2018.

### 2. Find employees with age between 30 and 40 (inclusive).

### 3. Find employees working in Seattle OR Boston.

In [0]:
df.filter(col('joining_date')<='2018-01-01').sort('joining_date').display()

In [0]:
df.filter((col('age') >=30) & (col('age')<=40)).sort('age').display()

In [0]:
df.filter(col('city').isin('Boston','Seattle')).sort(col('city')).display()

### 22. withColumnRenamed :

### 1. Rename salary → annual_salary.

### 2. Rename city → work_location.

In [0]:
df.withColumnRenamed('salary','annual_salary').display()

In [0]:
df.withColumnRenamed('city','work_location').display()

### 23. withColumn

### 1. Add a column half_salary = salary / 2.

### 2. Add a column name_length = length of full_name.

### 3. Add a column is_high_earner → "True" if salary > 80000 else "False".

In [0]:
df.withColumn('half_salary', col('salary')/2)

In [0]:
df_fullname = df.withColumn('full_name', concat_ws(' ', col('first_name'), col('last_name')))

In [0]:
df_length = df_fullname.withColumn('name_length', length(col('full_name')))

In [0]:
df_length.display()

In [0]:
df_length.withColumn('is_high_earner', when(col('salary')>80000, True).otherwise(False)).display()

### 24. Type Casting

### 1. Convert age to string.

### 2. Convert salary to integer.

In [0]:
df.withColumn('age', col('age').cast(IntegerType())).display()

In [0]:
df.withColumn('salary', col('salary').cast(IntegerType())).display()

### 25. Sort :

### 1. Sort employees by age descending.

### 2. Sort employees first by city, then first_name.

In [0]:
df.sort(col('age').desc()).display()

### 27. Limit :

### 1. Show top 4 employees with the earliest joining date.

### 2. Show 3 employees with the smallest salaries.

In [0]:
df.withColumn('joining_date', col('joining_date')).sort(col('joining_date')).limit(4).display()

In [0]:
df.withColumn('salary', col('salary')).sort(col('salary').asc()).limit(3).display()

### 27. Drop Duplicates :

### 1.Drop duplicate rows considering only department (keep one row per department).

### 2.Drop duplicates considering city and department together.

In [0]:
df.drop_duplicates(subset=['department']).display()

In [0]:
df.drop_duplicates(subset=['city','department']).display()

### 28. Bonus Combo :

### 1. Create a new column emp_summary in the format:
### "John Smith (IT) - Age: 29 - Salary: 75000.5".

### 2. Sort employees by salary and display the top 5 summaries.

In [0]:
df_summary = df_fullname.withColumn('emp_summary', concat_ws('',col('full_name'),lit(' '), lit('('), col('department'), lit(')'), lit(' - Age :'), col('age'), lit(' - Salary: '), col('salary') ))

In [0]:
df_summary.sort(col('salary').desc()).limit(5).display()